## Instagram Data 

## Data Ingetsion

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
BASE = '../Data/IGData.xlsx'

df = pd.read_excel(BASE)
df.head(10)

Tipo       Data  Curtidas  Comentários  Visualizações            Tags  \
0   Foto 2021-09-11      2858           16            NaN            Loja   
1   Foto 2021-09-11      2930           28            NaN   Loja/Produtos   
2   Foto 2021-09-11      2807            9            NaN            Loja   
3  Vídeo 2021-09-12      5115           49        82878.0        Produtos   
4   Foto 2021-09-13      4392           45            NaN        Produtos   
5   Foto 2021-09-17      5359           62            NaN  Novos Produtos   
6   Foto 2021-09-19     21597          852            NaN  Novos Produtos   
7   Foto 2021-09-25      6346           33            NaN        Produtos   
8   Foto 2021-09-27      6355           89            NaN        Produtos   
9  Vídeo 2021-09-28      4056           81        73174.0        Produtos   

  Pessoas Campanhas Carrossel  Interacoes   
0       N         N       NaN         2874  
1       N         N       NaN         2958  
2       N         N       NaN         2816  
3       N         N       NaN         5164  
4       S         N       NaN         4437  
5       N         S         S         5421  
6       S         S       NaN        22449  
7       N         S       NaN         6379  
8       S         N         S         6444  
9       N         N       NaN         4137

## Initial EDA

In [3]:
df.shape

(52, 10)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Tipo           52 non-null     object        
 1   Data           52 non-null     datetime64[ns]
 2   Curtidas       52 non-null     int64         
 3   Comentários    52 non-null     int64         
 4   Visualizações  15 non-null     float64       
 5   Tags           44 non-null     object        
 6   Pessoas        52 non-null     object        
 7   Campanhas      52 non-null     object        
 8   Carrossel      8 non-null      object        
 9   Interacoes     52 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 4.2+ KB


In [5]:
df.describe()

Data      Curtidas  Comentários  \
count                             52     52.000000    52.000000   
mean   2021-12-23 04:36:55.384615424  12262.730769   189.500000   
min              2021-09-11 00:00:00   2807.000000     9.000000   
25%              2021-10-23 06:00:00   5492.000000    69.500000   
50%              2022-01-05 00:00:00   9603.000000   128.000000   
75%              2022-02-15 12:00:00  17621.750000   265.250000   
max              2022-03-27 00:00:00  37351.000000   852.000000   
std                              NaN   8165.875326   170.687709   

       Visualizações   Interacoes   
count      15.000000     52.000000  
mean    53295.933333  12452.230769  
min     12627.000000   2816.000000  
25%     23625.000000   5562.500000  
50%     67205.000000   9773.500000  
75%     71686.000000  17920.750000  
max     92120.000000  37853.000000  
std     26975.183215   8299.390088

In [6]:
df.describe(include='object')

Tipo      Tags Pessoas Campanhas Carrossel
count     52        44      52        52         8
unique     4        12       2         2         1
top     Foto  Produtos       S         N         S
freq      36        12      40        30         8

In [7]:
df.isnull().sum()

Tipo              0
Data              0
Curtidas          0
Comentários       0
Visualizações    37
Tags              8
Pessoas           0
Campanhas         0
Carrossel        44
Interacoes        0
dtype: int64

In [8]:
#Separate into categorical and numerical
categorical_columns = [
    'Tags',
    'Pessoas',
    'Campanhas',
    'Carrossel'
]

numerical_columns = [
    'Curtidas',
    'Comentários',
    'Visualizações',
    'Interacoes'
]

In [9]:
for column in categorical_columns:
    print(f"Column: {column}")
    print(df[column].value_counts())
    print()

Column: Tags
Tags
Produtos                             12
Novos Produtos                        7
Influenciadores                       7
Datas comemorativas                   3
Dicas de como usar/Produtos           3
Trends                                3
Dicas de como usar/Novos Produtos     2
Loja                                  2
Promoções                             2
Loja/Produtos                         1
Trends/Produtos                       1
Datas comemorativas/Promoções         1
Name: count, dtype: int64

Column: Pessoas
Pessoas
S    40
N    12
Name: count, dtype: int64

Column: Campanhas
Campanhas
N    30
S    22
Name: count, dtype: int64

Column: Carrossel
Carrossel
S    8
Name: count, dtype: int64



In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Tipo           52 non-null     object        
 1   Data           52 non-null     datetime64[ns]
 2   Curtidas       52 non-null     int64         
 3   Comentários    52 non-null     int64         
 4   Visualizações  15 non-null     float64       
 5   Tags           44 non-null     object        
 6   Pessoas        52 non-null     object        
 7   Campanhas      52 non-null     object        
 8   Carrossel      8 non-null      object        
 9   Interacoes     52 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 4.2+ KB


In [11]:
print(df.duplicated().sum())

0


In [12]:
df.nunique()

Tipo              4
Data             50
Curtidas         52
Comentários      49
Visualizações    15
Tags             12
Pessoas           2
Campanhas         2
Carrossel         1
Interacoes       52
dtype: int64

## Basic Data Cleaning

### General Orientations
 - Understant where is the best engagement
 - Empty tags should be treated as empty
 - Ignore Visualization

In [13]:
# Renaming the columns from Portuguse to English
translation = {
    'Tipo':'Type',
    'Data':'Date',
    'Curtidas':'Likes',
    'Comentários': 'Comments',
    'Visualizações':'Views',
    'Pessoas':'People',
    'Campanhas': 'Campaign',
    'Carrossel':'Carousel',
    'Interacoes':'Interactions'
}

In [14]:
df = df.rename(columns=translation)
df.head()

Type       Date  Likes  Comments    Views           Tags People Campaign  \
0   Foto 2021-09-11   2858        16      NaN           Loja      N        N   
1   Foto 2021-09-11   2930        28      NaN  Loja/Produtos      N        N   
2   Foto 2021-09-11   2807         9      NaN           Loja      N        N   
3  Vídeo 2021-09-12   5115        49  82878.0       Produtos      N        N   
4   Foto 2021-09-13   4392        45      NaN       Produtos      S        N   

  Carousel  Interacoes   
0      NaN         2874  
1      NaN         2958  
2      NaN         2816  
3      NaN         5164  
4      NaN         4437

In [15]:
#As requested in the exercise, we need to drop the vizual;isation column
df = df.drop(columns='Views')
df.head()

Type       Date  Likes  Comments           Tags People Campaign Carousel  \
0   Foto 2021-09-11   2858        16           Loja      N        N      NaN   
1   Foto 2021-09-11   2930        28  Loja/Produtos      N        N      NaN   
2   Foto 2021-09-11   2807         9           Loja      N        N      NaN   
3  Vídeo 2021-09-12   5115        49       Produtos      N        N      NaN   
4   Foto 2021-09-13   4392        45       Produtos      S        N      NaN   

   Interacoes   
0         2874  
1         2958  
2         2816  
3         5164  
4         4437

In [16]:
insert_null = ['Carousel']

for column in insert_null:
    df.loc[df[column].isnull()] = df.loc[df[column].isnull()].fillna('N')

In [17]:
df.head()

Type       Date  Likes  Comments           Tags People Campaign Carousel  \
0   Foto 2021-09-11   2858        16           Loja      N        N        N   
1   Foto 2021-09-11   2930        28  Loja/Produtos      N        N        N   
2   Foto 2021-09-11   2807         9           Loja      N        N        N   
3  Vídeo 2021-09-12   5115        49       Produtos      N        N        N   
4   Foto 2021-09-13   4392        45       Produtos      S        N        N   

   Interacoes   
0         2874  
1         2958  
2         2816  
3         5164  
4         4437

In [18]:
df.isnull().sum()

Type           0
Date           0
Likes          0
Comments       0
Tags           0
People         0
Campaign       0
Carousel       0
Interacoes     0
dtype: int64

In [19]:
df.shape

(52, 9)

In [20]:
df['Tags'].value_counts()

Tags
Produtos                             12
N                                     8
Novos Produtos                        7
Influenciadores                       7
Datas comemorativas                   3
Dicas de como usar/Produtos           3
Trends                                3
Loja                                  2
Promoções                             2
Dicas de como usar/Novos Produtos     2
Loja/Produtos                         1
Trends/Produtos                       1
Datas comemorativas/Promoções         1
Name: count, dtype: int64